## Notebook 4: TTS Workflow

We have the exact podcast transcripts ready now to generate our audio for the Podcast.

In this notebook, we will learn how to generate Audio using both [lucasnewman/f5-tts-mlx](https://huggingface.co/lucasnewman/f5-tts-mlx) models first. 

After that, we will use the output from Notebook 3 to generate our complete podcast

In [3]:
import IPython.display as ipd
from f5_tts_mlx.generate import generate, SAMPLE_RATE
from tqdm import tqdm


### Testing the Audio Generation

Let's try generating audio using the models to understand how they work. 

#### F5 TTS MLX Model

Let's try using the Model first and generate a short segment voice.

In [2]:
TEST_AUDIO_FILE = "./resources/f5_tts_mlx_test_audio.wav"
MODEL = "lucasnewman/f5-tts-mlx"

In [3]:
# Define text and description
text_prompt = """
Exactly! And the distillation part is where you take a LARGE-model,and compress-it down into a smaller, more efficient model that can run on devices with limited resources.
"""

generate(
    generation_text=text_prompt,
    model_name=MODEL,
    output_path=TEST_AUDIO_FILE
)

In [4]:
# Play audio in notebook
ipd.Audio(TEST_AUDIO_FILE, rate=SAMPLE_RATE)

## Bringing it together: Making the Podcast

Okay now that we understand everything-we can now use the complete pipeline to generate the entire podcast

Let's load in our pickle file from earlier and proceed:

In [5]:
import pickle

with open('./resources/podcast_ready_data.pkl', 'rb') as file:
    PODCAST_TEXT = pickle.load(file)

We will concatenate the generated segments of audio and also their respective sampling rates since we will require this to generate the final audio

In [6]:
outputs = []

Function generate text for speaker 1

In [7]:
def generate_speaker1_audio(text, output_path):
    generate(
        generation_text=text,
        model_name=MODEL,
        output_path=output_path
    )

Function to generate text for speaker 2

In [8]:
def generate_speaker2_audio(text, output_path):
    generate(
        generation_text=text,
        model_name=MODEL,
        output_path=output_path,
        ref_audio_path="./resources/test_en_2_ref_short.wav",
        ref_audio_text="Some call me nature, others call me mother nature."
    )

Helper function to convert the numpy output from the models into audio

In [9]:
PODCAST_TEXT

'[\n    ("Speaker 1", "Alright folks, welcome to our podcast, where we dive deep into the cutting-edge world of Large Language Models (LLMs) and knowledge distillation. Today, we\'re exploring how we can enhance the capabilities of smaller models by transferring knowledge from larger, proprietary models. Imagine if we could make your humble assistant bot as smart as the mighty GPT-4! That\'s what this talk is all about. So, let\'s get started!"),\n    ("Speaker 2", "Wow, that sounds amazing! So, are we talking about making open-source models smarter just like the fancy proprietary ones?"),\n    ("Speaker 1", "Exactly! You got it. Open-source models are incredibly accessible, but they often don\'t have the depth and breadth of knowledge that proprietary models have. So, our goal is to bridge that gap and make them more powerful without needing to pay the hefty price tag."),\n    ("Speaker 2", "That\'s awesome! How do we do that? Is it like transferring skills or something?"),\n    ("Spe

Most of the times we argue in life that Data Structures isn't very useful. However, this time the knowledge comes in handy. 

We will take the string from the pickle file and load it in as a Tuple with the help of `ast.literal_eval()`

In [10]:
import ast

ast.literal_eval(PODCAST_TEXT)

[('Speaker 1',
  "Alright folks, welcome to our podcast, where we dive deep into the cutting-edge world of Large Language Models (LLMs) and knowledge distillation. Today, we're exploring how we can enhance the capabilities of smaller models by transferring knowledge from larger, proprietary models. Imagine if we could make your humble assistant bot as smart as the mighty GPT-4! That's what this talk is all about. So, let's get started!"),
 ('Speaker 2',
  'Wow, that sounds amazing! So, are we talking about making open-source models smarter just like the fancy proprietary ones?'),
 ('Speaker 1',
  "Exactly! You got it. Open-source models are incredibly accessible, but they often don't have the depth and breadth of knowledge that proprietary models have. So, our goal is to bridge that gap and make them more powerful without needing to pay the hefty price tag."),
 ('Speaker 2',
  "That's awesome! How do we do that? Is it like transferring skills or something?"),
 ('Speaker 1',
  "Yes, it'

#### Generating the Final Podcast

Finally, we can loop over the Tuple and use our helper functions to generate the audio

In [13]:
final_audio = None

i = 1

for speaker, text in tqdm(ast.literal_eval(PODCAST_TEXT), desc="Generating podcast segments", unit="segment"):
    output_path = f"./resources/segments/_podcast_segment_{i}.wav"
    if speaker == "Speaker 1":
        generate_speaker1_audio(text, output_path)
    else:  # Speaker 2
        generate_speaker2_audio(text, output_path)
    i += 1

Generating podcast segments:   0%|          | 0/38 [00:00<?, ?segment/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Got reference audio with duration: 5.33 seconds
Got duration of 2897 frames (24.92368507385254 secs) for generated speech.


Generating podcast segments:   3%|▎         | 1/38 [00:59<36:31, 59.23s/segment]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Got reference audio with duration: 3.94 seconds
Got duration of 598 frames (5.146483898162842 secs) for generated speech.


Generating podcast segments: 100%|██████████| 38/38 [17:58<00:00, 28.39s/segment]

Generated speech in 0:00:09.115681
Generated 2.47 seconds of audio in 0:00:09.115826.


In [1]:
# Combine the segments ./resources/segments
import re
import os
import numpy as np
import soundfile as sf

audio_files = sorted([f"./resources/segments/{file}" for file in os.listdir("./resources/segments")],
                     key=lambda x: int(re.search(r'segment_(\d+)\.wav', x).group(1)))

print("audio_files -> ", audio_files)
audio_data = []
for file in audio_files:
    data, rate = sf.read(file)
    audio_data.append(data)

audio_data = np.concatenate(audio_data)

audio_files ->  ['./resources/segments/_podcast_segment_1.wav', './resources/segments/_podcast_segment_2.wav', './resources/segments/_podcast_segment_3.wav', './resources/segments/_podcast_segment_4.wav', './resources/segments/_podcast_segment_5.wav', './resources/segments/_podcast_segment_6.wav', './resources/segments/_podcast_segment_7.wav', './resources/segments/_podcast_segment_8.wav', './resources/segments/_podcast_segment_9.wav', './resources/segments/_podcast_segment_10.wav', './resources/segments/_podcast_segment_11.wav', './resources/segments/_podcast_segment_12.wav', './resources/segments/_podcast_segment_13.wav', './resources/segments/_podcast_segment_14.wav', './resources/segments/_podcast_segment_15.wav', './resources/segments/_podcast_segment_16.wav', './resources/segments/_podcast_segment_17.wav', './resources/segments/_podcast_segment_18.wav', './resources/segments/_podcast_segment_19.wav', './resources/segments/_podcast_segment_20.wav', './resources/segments/_podcast_s

### Output the Podcast

We can now save this as a wav file

In [4]:
sf.write("./resources/_podcast.wav", audio_data, SAMPLE_RATE)

### Suggested Next Steps:

- Experiment with the prompts: Please feel free to experiment with the SYSTEM_PROMPT in the notebooks
- Extend workflow beyond two speakers
- Test other TTS Models
- Experiment with Speech Enhancer models as a step 5.

In [ ]:
#fin